<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #6: Word2Vec - Jhonatahn Giraldo
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`



## Libre elección

Expectativas:
- Uso de Word2Vec 
- Responder:
    - ¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?
    - ¿Qué problemas encontró al realizar este taller?
    
    
Ideas algunas por si están varados: 
- Hacer más visualizaciones
- Jugar con los parámetros de la funcion word2vec de gensim
- Usar alguna función de gensim que no hayamos visto en clase (https://radimrehurek.com/gensim/models/word2vec.html#module-gensim.models.word2vec)
- Comparar palabras del inglés y el español
- Encontrar analogías y similutdes interesantes


### Enviar código con explicaciones del por qué escogieron ese camino

In [15]:
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

import gensim.models.word2vec as w2v
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

In [16]:
data = pd.read_csv("clubdelapelea.csv",sep = ";")
data

,personaje,dialogo
0,Narrador,"Cuando compras muebles, te dices, ya está. Ya ..."
1,Tyler Durden,"Joder, tío, y ahora nada."
2,Narrador,Nada de nada. Nada
3,Tyler Durden,¿Sabes lo qué es un edredón nórdico?
4,Narrador,Un cobertor.
5,Tyler Durden,Exacto. Es una manta. Sólo una manta. ¿Por qué...
6,Narrador,No sé. Consumidores.
7,Tyler Durden,Exacto. Consumidores. Somos subproductos de un...
8,Tyler Durden,Has perdido un montón de soluciones para la vi...
9,Tyler Durden,“Las cosas que posees terminan poseyéndote”.


In [17]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()

data['pp'] = data['dialogo'].apply(lambda texto: pre_procesado(texto))

data

,personaje,dialogo,pp
0,Narrador,"Cuando compras muebles, te dices, ya está. Ya ...","[compras, muebles, dices, nunca, necesitaré, s..."
1,Tyler Durden,"Joder, tío, y ahora nada.","[joder, tío, ahora]"
2,Narrador,Nada de nada. Nada,[]
3,Tyler Durden,¿Sabes lo qué es un edredón nórdico?,"[sabes, edredón, nórdico]"
4,Narrador,Un cobertor.,[cobertor]
5,Tyler Durden,Exacto. Es una manta. Sólo una manta. ¿Por qué...,"[exacto, manta, sólo, manta, sabemos, tíos, ed..."
6,Narrador,No sé. Consumidores.,"[sé, consumidores]"
7,Tyler Durden,Exacto. Consumidores. Somos subproductos de un...,"[exacto, consumidores, subproductos, obsesión,..."
8,Tyler Durden,Has perdido un montón de soluciones para la vi...,"[perdido, montón, soluciones, vida, moderna]"
9,Tyler Durden,“Las cosas que posees terminan poseyéndote”.,"[cosas, posees, terminan, poseyéndote]"


In [37]:
modelop = w2v.Word2Vec(data['pp'].values,
                            sg=1, # 1 para usar skip-gram, 0 para usar CBOW
                            seed=1, # 1 para tener resultados reproducibles y debugear
                            size=3, # el tamaño de los vectores de palabras, o neuronas
                            min_count=2, #mínimo de veces que ha de aparecer cada palabra, para ser considerada,
                            window=4) # ventana contextual de cada palabra

In [38]:
len(modelop.wv.vocab.keys())

9

In [39]:
X = modelop[modelop.wv.vocab]
matrix = pd.DataFrame(X)
matrix.index = modelop.wv.vocab.keys()

C:\Users\jagir\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [40]:
pca = PCA(n_components=2)

result = pca.fit_transform(matrix)
result = pd.DataFrame(result)
result.columns = ['X', 'Y']
result['Palabra'] = matrix.index.values
personaje = pre_procesado(data.personaje.str.cat(sep=" "))
result['Color'] = result['Palabra'].apply(lambda val: "#f8db27" if val in personaje else "#2f64d6")


trace_palabras = go.Scatter(x=result[result['Color']=='#2f64d6']['X'].values,
                               y=result[result['Color']=='#2f64d6']['Y'].values,
                               text=result[result['Color']=='#2f64d6']['Palabra'].values,
                               mode='markers',
                               marker={'color': result[result['Color']=='#2f64d6']['Color'].values, 'size':10},
                               name='Palabras') 

trace_personaje = go.Scatter(x=result[result['Color']=='#f8db27']['X'].values,
                               y=result[result['Color']=='#f8db27']['Y'].values,
                               text=result[result['Color']=='#f8db27']['Palabra'].values,
                               mode='markers',
                               marker={'color': result[result['Color']=='#f8db27']['Color'].values, 'size':10},
                               name='personaje') 

layout = go.Layout(title="PCA")

fig = go.Figure(data=[trace_palabras,trace_personaje], layout=layout)
iplot(fig)

In [41]:
modelop.wv.most_similar("manta")

[('consumidores', 0.7789719104766846),
 ('cosas', 0.4861029088497162),
 ('sofá', 0.43656125664711),
 ('etc', 0.11531957983970642),
 ('vida', 0.049501627683639526),
 ('pase', -0.01404029130935669),
 ('exacto', -0.2054252028465271),
 ('edredón', -0.3925222158432007)]

In [42]:
modelop.similarity('manta', 'edredón')

C:\Users\jagir\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).



-0.39252222

### Respuestas : El modelo no arroja buenos resultados puesto que al entender el contexto del dialogo se relaciona totalmente un edredón con una manta y segun el modelo estas palbras no tienen relacion alguna

### El motivo principal pudo ser por que el texto es muy corto y la bolsa de palabras no es lo suficientemente grande 

### Modelar con el dataset tan pequeño puede volverse un problema, otro que tuve es que la estructura cuando es un texto plano es dificil transformarlo, por ejemplo una cancion tomada de google